# Dog Breed Identification

**Imports:**

In [1]:
import keras
keras.__version__

C:\Users\Jagan\Anaconda3\envs\deep-learning\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


'2.0.9'

In [2]:
import os, shutil
import pandas as pd
import numpy as np

from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Dropout, Flatten, Input, Conv2D, MaxPool2D
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.optimizers import RMSprop
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
from keras.applications import VGG16, VGG19, ResNet50
from keras.layers.pooling import GlobalAveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Activation
from keras.callbacks import ModelCheckpoint

import matplotlib.pyplot as plt
%matplotlib inline

# custom imports
from utils import my_utils

%reload_ext autoreload
%autoreload 2

In [4]:
# Operating System
import platform
print(platform.system(), platform.release())

Windows 10


**Note:** The paths below have been written in 'Windows' format. To have cross-platform compatibility (for which I didn't bother) you need to covert the paths into platform dependent format.

**Directories/Paths:**

In [5]:
base_dir = '../datasets/dog_breed_identification'

In [6]:
print(*os.listdir(base_dir), sep = "\t")

bin	labels.csv	sample_submission.csv	test	train


In [7]:
print(*os.listdir(os.path.join(base_dir,'bin')), sep = "\t")

train	train_map.csv	val	val_map.csv


In [8]:
train_dir = os.path.join(base_dir, 'bin', 'train')
val_dir = os.path.join(base_dir, 'bin', 'val')
test_dir = os.path.join(base_dir, 'test')

In [9]:
# directory to save models
save_dir = '../saved_models/dog_breed_identification/2'
if not os.path.isdir(save_dir): os.makedirs(save_dir)

**Data Parameters:**

In [10]:
img_rows, img_cols, img_chnls = 224, 224, 3
input_shape = (img_rows,img_cols,img_chnls)
target_size = input_shape[:-1] # (224, 224)

num_classes = 120

train_sample_count = 8177 
val_sample_count = 2045
test_sample_count = len(os.listdir(test_dir))
batch_size = 32

**Pre-Trained ConvNet:** 

In [11]:
# vgg16 convbase
vgg16_conv_base = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
# vgg16_conv_base.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [12]:
# vgg19 convbase
vgg19_conv_base = VGG19(weights='imagenet', include_top=False, input_shape=input_shape)
# vgg19_conv_base.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [13]:
# resnet50 convbase
resnet50_convbase = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
# resnet50_convbase.summary()

A local file was found, but it seems to be incomplete or outdated because the md5 file hash does not match the original value of a268eb855778b3df3c7506639542a6af so we will re-download the data.
94658560/94653016 [==============================] - 511s 5us/step
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        input_3[0][0]                    
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
_____________________________________________

**Fast Feature Extraction:**

In [14]:
def input_branch(input_shape=None):
    size = int(input_shape[2] / 4)
    
    branch_input = Input(shape=input_shape)
    
    branch = GlobalAveragePooling2D()(branch_input)
    branch = Dense(size, use_bias=False, kernel_initializer='uniform')(branch)
    branch = BatchNormalization()(branch)
    
    branch_output = Activation("relu")(branch)
    
    return branch_output, branch_input

In [15]:
vgg16_input_shape = ((vgg16_conv_base.layers[-1]).output_shape)[1:]
vgg19_input_shape = ((vgg19_conv_base.layers[-1]).output_shape)[1:]
resnet50_input_shape = ((resnet50_convbase.layers[-1]).output_shape)[1:]

In [16]:
# Sanity Check
vgg16_input_shape, vgg19_input_shape, resnet50_input_shape

((7, 7, 512), (7, 7, 512), (1, 1, 2048))

In [17]:
vgg16_output, vgg16_input = input_branch(input_shape = vgg16_input_shape)
vgg19_output, vgg19_input = input_branch(input_shape = vgg19_input_shape)
resnet50_output, resnet50_input = input_branch(input_shape= resnet50_input_shape)

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [32]:
input_branches = Concatenate()([vgg16_output, vgg19_output, resnet50_output])
net = Dropout(0.3)(input_branches)
net = Dense(600, use_bias=False, kernel_initializer='uniform')(net)
net = BatchNormalization()(net)
net = Activation("relu")(net)
net = Dropout(0.3)(net)
net = Dense(120, kernel_initializer='uniform', activation="softmax")(net)

model = Model(inputs=[vgg16_input, vgg19_input, resnet50_input], outputs=[net])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 7, 7, 512)    0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            (None, 7, 7, 512)    0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            (None, 1, 1, 2048)   0                                            
__________________________________________________________________________________________________
global_average_pooling2d_1 (Glo (None, 512)          0           input_4[0][0]                    
__________________________________________________________________________________________________
global_ave

In [ ]:
# data generators
train_generator = my_utils.data_generator(train_dir, target_size, batch_size)
val_generator = my_utils.data_generator(val_dir, target_size, batch_size)

In [ ]:
# extract features vgg16
vgg16_train_features, vgg16_train_labels = my_utils.extract_features(vgg16_conv_base, train_sample_count, 
                                                           train_generator, num_classes, batch_size)
vgg16_val_features, vgg16_val_labels = my_utils.extract_features(vgg16_conv_base, val_sample_count, 
                                                       val_generator, num_classes, batch_size)

In [ ]:
# extract features vgg19
vgg19_train_features, vgg19_train_labels = my_utils.extract_features(vgg19_conv_base, train_sample_count, 
                                                           train_generator, num_classes, batch_size)
vgg19_val_features, vgg19_val_labels = my_utils.extract_features(vgg19_conv_base, val_sample_count, 
                                                       val_generator, num_classes, batch_size)

In [ ]:
# extract features resnet 50
resnet50_train_features, resnet50_train_labels = my_utils.extract_features(resnet50_convbase, train_sample_count, 
                                                           train_generator, num_classes, batch_size)
resnet50_val_features, resnet50_val_labels = my_utils.extract_features(resnet50_convbase, val_sample_count, 
                                                       val_generator, num_classes, batch_size)

In [39]:
a = vgg16_train_labels
b = vgg19_train_labels
c = resnet50_train_labels

(a == b).all() and (b == c).all()

True

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer="rmsprop", metrics=['accuracy'])

checkpointer = ModelCheckpoint(filepath= os.path.join(save_dir, 'best_model.h5'), verbose=1, save_best_only=True)

history = model.fit([vgg16_train_features, vgg19_train_features, resnet50_train_features], vgg16_train_labels, 
                  validation_data=([vgg16_val_features, vgg19_val_features, resnet50_val_features], vgg16_val_labels),
                  epochs=500, batch_size=batch_size, callbacks=[checkpointer], verbose=1)

In [ ]:
# save moodel
model_name = 'dbi_adv_3.h5'
model.save(os.path.join(save_dir, model_name))

In [ ]:
# plot model history
my_utils.plot_history(history)